In [5]:
import pandas as pd
from IPython.display import display  # built-in Jupyter

In [6]:
pd.__version__ 

'2.3.2'

In [7]:
df = pd.read_csv("car_fuel_efficiency.csv") 
num_rows = df.shape[0] 
print(f"Rows (records): {num_rows}")

Rows (records): 9704


In [8]:
# Try to detect the fuel-type column automatically
candidates = [c for c in df.columns if "fuel" in c.lower()]
selected_col = None
reason = ""

if len(candidates) == 1:
    selected_col = candidates[0]
    reason = f"Detected single column containing 'fuel': '{selected_col}'"
elif len(candidates) > 1:
    # Prefer columns that look like exact fuel type labels
    preferred = [c for c in candidates if "type" in c.lower() or c.lower().strip() in {"fuel", "fueltype", "fuel_type", "fuel type"}]
    if preferred:
        selected_col = preferred[0]
        reason = f"Multiple 'fuel' columns found; chose '{selected_col}' as it looks like a fuel type column."
    else:
        selected_col = candidates[0]
        reason = f"Multiple 'fuel' columns found; defaulting to first match: '{selected_col}'."
else:
    # No column includes 'fuel' in its name; fall back to common names
    common_names = ["Fuel Type", "fuelType", "FuelType", "fuel_type", "fueltype", "fuel"]
    for name in common_names:
        if name in df.columns:
            selected_col = name
            reason = f"No 'fuel' substring match; fell back to common name: '{selected_col}'"
            break

# Prepare outputs
if selected_col is None:
    result = {
        "status": "no_fuel_column_found",
        "message": "Couldn't automatically find a column for fuel type. Here are the columns.",
        "columns": list(df.columns),
    }
    print(result["message"])
    print("Columns:", result["columns"])
else:
    # Count unique fuel types (excluding NaN)
    unique_vals = sorted(v for v in df[selected_col].dropna().unique())
    nunique = len(unique_vals)
    print(f"Fuel-type column: {selected_col}")
    print(reason)
    print(f"Number of distinct fuel types: {nunique}")
    print("Fuel types:", unique_vals)

    # Also show a frequency table for transparency
    counts = df[selected_col].value_counts(dropna=True).rename_axis("fuel_type").reset_index(name="count")

Fuel-type column: fuel_type
Multiple 'fuel' columns found; chose 'fuel_type' as it looks like a fuel type column.
Number of distinct fuel types: 2
Fuel types: ['Diesel', 'Gasoline']


In [9]:
# Count missing values per column
na_counts = df.isna().sum()

# Keep only columns that have at least one missing value
cols_with_na = na_counts[na_counts > 0].sort_values(ascending=False)

print(f"Columns with missing values: {cols_with_na.size} / {df.shape[1]}")

# Show a small table with counts and percentages (if there are any)
if cols_with_na.size:
    summary = cols_with_na.to_frame("missing_count")
    summary["missing_pct"] = (summary["missing_count"] / len(df) * 100).round(2)
    display(summary)
else:
    print("No missing values in any column.")

Columns with missing values: 4 / 11


,missing_count,missing_pct
acceleration,930,9.58
horsepower,708,7.30
num_doors,502,5.17
num_cylinders,482,4.97


In [10]:
def pick_region_col(cols):
    keys = ["origin", "region", "country", "market", "continent", "manufacturer_country", "made_in"]
    cands = [c for c in cols if any(k in c.lower() for k in keys)]
    return cands[0] if cands else None

def pick_eff_col(cols):
    scored = []
    for c in cols:
        cl = c.lower()
        if any(k in cl for k in ["mpg", "kmpl", "km/l", "l/100", "fuel_eff", "fuel eff", "efficiency", "consumption"]):
            score = 1
            if "combined" in cl or "overall" in cl or "avg" in cl: score += 3
            if "mpg" in cl: score += 2
            # deprioritize city/highway-only if a combined exists
            if "city" in cl or "urban" in cl: score -= 0
            if "highway" in cl or "extra" in cl: score -= 0
            scored.append((score, c))
    if scored:
        scored.sort(reverse=True)
        return scored[0][1]
    return None

region_col = pick_region_col(df.columns)
eff_col = pick_eff_col(df.columns)

if region_col is None or eff_col is None:
    print("Couldn't auto-detect needed columns.")
    print("Columns:", list(df.columns))
else:
    # Filter rows where region contains 'asia' (case-insensitive)
    asia_mask = df[region_col].astype(str).str.contains("asia", case=False, na=False)
    asia_df = df.loc[asia_mask].copy()

    # Ensure numeric efficiency
    asia_df[eff_col] = pd.to_numeric(asia_df[eff_col], errors="coerce")

    if asia_df.empty or asia_df[eff_col].dropna().empty:
        print(f"No Asian cars or '{eff_col}' has no numeric values.")
    else:
        max_eff = asia_df[eff_col].max()
        print(f"Region column: {region_col}")
        print(f"Efficiency column: {eff_col}")
        print(f"Maximum fuel efficiency (Asia): {max_eff}")

        # Show rows achieving the max (top 10 if many)
        top = asia_df.loc[asia_df[eff_col] == max_eff]
        display(top.head(10))

Region column: origin
Efficiency column: fuel_efficiency_mpg
Maximum fuel efficiency (Asia): 23.759122836520497


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
9387,330,3.0,136.0,1223.298226,NaN,2001,Asia,Gasoline,Front-wheel drive,1.0,23.759123


In [11]:
hp_col = "horsepower" 
median_hp = pd.to_numeric(df[hp_col], errors="coerce").median()
print("Median horsepower:", median_hp)

Median horsepower: 149.0


In [12]:

# Convert to numeric and drop missing values
s = pd.to_numeric(df[hp_col], errors="coerce").dropna()

# Count frequencies
counts = s.value_counts()

if counts.empty:
    print("No numeric horsepower values found.")
else:
    max_count = int(counts.max())
    modes = counts[counts == max_count].index.tolist()
    print(f"Most frequent horsepower value(s): {modes} (count = {max_count})")

    # Optional: show the top 10 most common horsepower values
    counts.head(10)

Most frequent horsepower value(s): [152.0] (count = 142)


In [13]:
# Make sure the column is numeric (strings like "150 hp" -> NaN)
s = pd.to_numeric(df[hp_col], errors="coerce")

# Most frequent value (mode). If there are multiple modes, take the first.
mode_hp = s.mode(dropna=True).iloc[0]

# Fill NaNs with the mode and write back
n_missing_before = s.isna().sum()
df[hp_col] = s.fillna(mode_hp)

print(f"Filled {n_missing_before} missing values in '{hp_col}' with mode = {mode_hp}")

# (Optional) If all values are whole numbers, store as nullable integer
if (df[hp_col] % 1 == 0).all():
    df[hp_col] = df[hp_col].astype("Int64")

Filled 708 missing values in 'horsepower' with mode = 152.0


In [14]:
median_hp = pd.to_numeric(df[hp_col], errors="coerce").median()
print("Median horsepower:", median_hp)

Median horsepower: 152.0
